# Machine Learning Programming Exercise
An Analytical Methods in Cancer Genomics project by Lukáš Cakl `cakll@natur.cuni.cz`.

## Step 1 - fetching the data
Here our dataset will be explained, fetched, read and split into its main components.

The dataset available was provided as part of the exercise [here](https://gear.embl.de/data/.slides/deletion.tsv.gz). It contains the following features:
- size: Size of the deletion
- vac: How many alleles have the deletion (2504 diploid samples -> 5008 alleles in total)
- vaf: Allele frequency (simply vac/5008)
- pass: 1 - Deletion prediction passed all algorithm filters, 0 – deletion did not pass all filters
- missingrate: Fraction of samples not genotyped for this deletion
- precise: 1 - The deletion has single-nucleotide resolution, 0 - only approximate breakpoints are known
- ci: Confidence interval size around the deletion breakpoints (how sure is the algorithm about the breakpoint)
- ce: DNA sequence entropy of the deletion
- refgq: Average genotype quality of homozygous reference samples
- altgq: Average genotype quality of deletion carrier samples
- rdratio: Read-depth ratio of deletion carriers compared to deletion non-carriers
- altratio: Average fraction of reads supporting the deletion in a carrier sample
- refratio: Average fraction of reads supporting the deletion in a non-carrier sample
- maxaltratio: Maximum fraction of reads supporting the deletion in a carrier sample
- status: 1 – likely true deletion, 0 – likely false deletion, NA – unlabeled (deletion could be true or false)

To download the data we use urllib. We also check if it already exists localy to save on bandwidth.

In [1]:
import urllib
import os

datasetUrl = 'https://gear.embl.de/data/.slides/deletion.tsv.gz'
def downloadData(sourceUrl, localFile, localPath):
    if not os.path.isdir(localPath):
        os.makedirs(localPath)
    localUri = os.path.join(localPath, localFile)
    if not os.path.exists(localUri):
        urllib.request.urlretrieve(sourceUrl, localUri)
        print('Dataset downloaded')
    else:
        print('No download neccessary')
downloadData(datasetUrl, 'deletion.tsv.gz', './data')

No download neccessary


Then we parse the dataset into a pandas dataframe.

In [2]:
import pandas as pd
dataset = pd.read_csv("./data/deletion.tsv.gz", delimiter='\t', compression='gzip')
dataset

,chr_start_end,id,size,vac,vaf,pass,missingrate,precise,ci,ce,refgq,altgq,rdratio,refratio,altratio,maxaltratio,status
0,chr1_597938_598745,DEL00000012,808,1,0.000200,0,0.001997,0,287,0.00000,26,11,0.945455,0.0,0.285714,0.285714,NaN
1,chr1_713026_713083,DEL00000018,58,7,0.001399,1,0.000799,1,6,1.90770,21,28,0.648186,0.0,0.500000,0.888889,NaN
2,chr1_778727_778781,DEL00000020,55,1,0.000200,1,0.000000,1,2,1.92127,96,10000,0.668154,0.0,0.547619,0.547619,NaN
3,chr1_938037_938670,DEL00000080,634,10,0.001997,1,0.000000,1,5,1.92573,102,10000,0.434565,0.0,0.545455,1.000000,NaN
4,chr1_1053192_1053244,DEL00000114,53,307,0.061302,1,0.000000,1,24,1.87118,78,10000,0.717275,0.0,0.590909,1.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27100,chrX_155250109_155250983,DEL00202501,875,10,0.001997,1,0.000000,0,136,0.00000,54,21,0.964944,0.0,0.090909,1.000000,NaN
27101,chrX_155265807_155266142,DEL00202508,336,2,0.000399,1,0.000000,1,5,1.88236,48,10000,0.522206,0.0,0.636364,0.636364,NaN
27102,chrX_155666910_155667807,DEL00202560,898,1,0.000200,1,0.000000,1,5,1.90604,54,135,0.499575,0.0,0.291667,0.291667,NaN
27103,chrX_155819916_155819986,DEL00202579,71,18,0.003594,1,0.000000,1,8,1.98825,81,10000,0.627721,0.0,0.536585,0.884615,NaN


We trim out the chr_start_end and id columns, which will not be used in the machine learning parts. We save the id for later so we can output our predictions on a per-id basis.

In [3]:
ids = dataset["id"]
dataset = dataset.drop(["chr_start_end", "id"], axis=1)

And we split the unknown data (status NaN) from the data we will use for training and testing the model.
We also drop the target column and save it separately

In [4]:
import numpy as np

unknownDataset = dataset[np.isnan(dataset["status"])]
unknownDataset = unknownDataset.drop(["status"], axis=1)
unknownIds = ids[np.isnan(dataset["status"])]
knownDataset = dataset[np.invert(np.isnan(dataset["status"]))]
knownTarget = knownDataset["status"]
knownDataset = knownDataset.drop(["status"], axis=1)
knownIds = ids[np.invert(np.isnan(dataset["status"]))]

## Step 2 - dataset exploration
In this step we explore the available dataset using describe and PCA. This is just to get a handle on the data and see if some feature engineering is required (e.q. scaling certain values).

In [5]:
knownDataset.describe()

,size,vac,vaf,pass,missingrate,precise,ci,ce,refgq,altgq,rdratio,refratio,altratio,maxaltratio
count,1439.000000,1439.000000,1439.000000,1439.000000,1439.000000,1439.000000,1439.000000,1439.000000,1439.000000,1439.000000,1439.000000,1439.000000,1439.000000,1439.000000
mean,284.558721,183.475330,0.037053,0.965949,0.002652,0.876998,21.333565,1.660749,89.348158,7976.104934,0.633950,0.001194,0.477501,0.609359
std,270.905760,578.185304,0.116512,0.181424,0.034514,0.328554,55.636236,0.632800,370.364053,4004.093931,0.172112,0.009814,0.159454,0.224770
min,52.000000,1.000000,0.000200,0.000000,0.000000,0.000000,1.000000,0.000000,5.000000,3.000000,0.003728,0.000000,0.055556,0.066667
25%,75.000000,1.000000,0.000200,1.000000,0.000000,1.000000,3.000000,1.822565,72.000000,10000.000000,0.528501,0.000000,0.437500,0.478261
50%,148.000000,3.000000,0.000599,1.000000,0.000000,1.000000,4.000000,1.915120,78.000000,10000.000000,0.600852,0.000000,0.500000,0.571429
75%,442.000000,26.000000,0.005391,1.000000,0.000000,1.000000,8.000000,1.958650,84.000000,10000.000000,0.686155,0.000000,0.562500,0.690065
max,998.000000,4517.000000,0.901957,1.000000,0.821885,1.000000,514.000000,1.999220,10000.000000,10000.000000,1.842420,0.157895,1.000000,1.000000


Just from the dataset description we can see many features (size, vac, refgq...) which are on a vastly larger scale than e.g. vaf. Such features will probably require scaling to not overshadow others.

### PCA
PCA is an useful method of dimensionality reduction, which I will be using to explore the dataset

In [6]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pcaKnownDataset = pca.fit_transform(knownDataset)
print(pca.components_)
print(pca.explained_variance_ratio_)

[[ 5.45354620e-03  3.96215082e-02  8.11860835e-06 -1.46790350e-05
   1.20131552e-06 -2.37440708e-05  4.97058527e-03 -5.42778322e-05
   4.40738946e-03 -9.99177791e-01  2.27332087e-05  3.82650081e-07
  -1.81833548e-05 -8.26336384e-06]
 [-8.75473606e-02  9.93719099e-01  1.99582921e-04  4.26724008e-05
   3.48124989e-07  8.67784810e-05 -1.36484340e-02  8.98604862e-05
  -5.63972097e-02  3.86105378e-02  2.99460875e-05  7.41475536e-06
   1.04232222e-04  2.10366689e-04]]
[0.96851554 0.01881614]


In [7]:
import matplotlib.pyplot as plt
%matplotlib widget
fig, ax = plt.subplots()
ax.scatter(pcaKnownDataset[knownTarget == 0][:,0], pcaKnownDataset[knownTarget == 0][:,1], label="0")
ax.scatter(pcaKnownDataset[knownTarget == 1][:,0], pcaKnownDataset[knownTarget == 1][:,1], label="1")
ax.legend(bbox_to_anchor=(0, 1, 1, 0), loc="lower left", mode="expand", ncol=2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In PCA we can see that most of the variance is in the altgq feature, which is out of scale. While the graph looks nicely split, zooming in on certain parts shows that the values are in fact intermixed. We also can one hot encode the categorical pass and precise

In [8]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([
        ("standardScale", StandardScaler(), ['size', 'vac', 'ci', 'ce', 'refgq', 'altgq']),
        ("oneHot", OneHotEncoder(handle_unknown='ignore'), ['pass', 'precise'])
    ], remainder='passthrough')
scaledKnownDataset = ct.fit_transform(knownDataset)

In [9]:
pcaScaled = PCA(n_components=2)
pcaScaledKnownDataset = pcaScaled.fit_transform(scaledKnownDataset)
print(pcaScaled.components_)
print(pcaScaled.explained_variance_ratio_)

[[ 4.23599301e-01  2.56070375e-02  5.52876951e-01 -5.78214513e-01
   1.44494435e-02 -3.12647426e-01  5.07089688e-02 -5.07089688e-02
   1.88653629e-01 -1.88653629e-01  3.03395369e-03  3.44958955e-04
   3.33649445e-02  1.43285808e-04 -5.56863030e-02 -3.74039560e-02]
 [-3.31583142e-01  7.51404979e-01 -3.13276370e-02  3.34552141e-02
  -4.07456457e-02 -5.50091496e-01  8.86284452e-03 -8.86284452e-03
  -3.70056010e-02  3.70056010e-02  8.76262874e-02  3.31654785e-03
   6.48034783e-02  3.49420262e-03  8.09255590e-03  6.49550569e-02]]
[0.40065605 0.19902903]


In [10]:
fig, ax = plt.subplots()
ax.scatter(pcaScaledKnownDataset[knownTarget == 0][:,0], pcaScaledKnownDataset[knownTarget == 0][:,1], label="0")
ax.scatter(pcaScaledKnownDataset[knownTarget == 1][:,0], pcaScaledKnownDataset[knownTarget == 1][:,1], label="1")
ax.legend(bbox_to_anchor=(0, 1, 1, 0), loc="lower left", mode="expand", ncol=2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

We see that less variance is explained by a single big value and that the graph seems much less intermixed, which is a good indication that the feature enginnering done above helps.

## Step 3 - constructing a scikit-learn pipeline
In this step we will use a scikit-learn pipeline to do our feature engineering, training and model selection for us. Here I will also split out a training portion of the dataset, which will be used for the pipeline and a test portion to measure the final results.

A random_state seed of 42 will be used to provide deterministic results in this notebook.

In [15]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold

trainData, testData, trainTarget, testTarget = train_test_split(knownDataset, knownTarget, stratify = knownTarget, random_state=42)

In [30]:
from sklearn.pipeline import Pipeline
from pipelinehelper import PipelineHelper
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier

pipeline = Pipeline([
            ('featureEngineering', ct),
            ('classifier', PipelineHelper([
                ('tree', DecisionTreeClassifier(random_state=42)),
                ('randomForest', RandomForestClassifier(random_state=42)),
                ('gradientBoosting', GradientBoostingClassifier(random_state=42)),
                ('svm', SVC(random_state=42)),
                ('mlp', MLPClassifier(random_state=42, max_iter = 500)),
                ('knn', KNeighborsClassifier()),
                ('nb', GaussianNB()),
                ('gp', GaussianProcessClassifier()),
            ]))
        ])
params = {
    'classifier__selected_model': pipeline.named_steps['classifier'].generate({
        'tree__criterion': ['gini', 'entropy'],
        'tree__splitter': ['best', 'random'],
        'tree__max_depth': np.linspace(5, 15, 11),
        
        'randomForest__n_estimators': [75, 100, 125],
        'randomForest__criterion': ['gini', 'entropy'],
        'randomForest__max_depth': [3, 5, 8, 10],
        'randomForest__class_weight': ['balanced', 'balanced_subsample'],
        
        'gradientBoosting__loss': ['deviance', 'exponential'],
        'gradientBoosting__learning_rate': [0.1, 0.05, 0.01],
        'gradientBoosting__n_estimators': [100, 125, 150],
        'gradientBoosting__max_depth': [2, 3, 5],
        'gradientBoosting__subsample': [0.5 , 0.75, 1],
        'gradientBoosting__max_features': [None, 'auto'],
        
        'svm__C': [0.5, 1, 2],
        'svm__kernel': ['linear', 'rbf', 'sigmoid'],
        'svm__gamma': ['scale', 'auto'],
        'svm__decision_function_shape': ['ovo', 'ovr'],
        
        'mlp__hidden_layer_sizes': [(20), (50), (10, 5), (20, 10)],
        'mlp__activation': ['relu', 'logistic'],
        'mlp__learning_rate': ['constant', 'adaptive', 'invscaling'],
        'mlp__alpha': [0.001, 0.0001, 0.00001],
        
        'knn__n_neighbors': [3, 5, 10],
        'knn__weights': ['uniform', 'distance'],
    }),
}
crossValidation = StratifiedKFold(n_splits = 5)
grid = GridSearchCV(pipeline, params, scoring='f1', n_jobs = 5, cv = crossValidation, verbose = 4)

In [31]:
grid.fit(trainData, trainTarget)

Fitting 5 folds for each of 532 candidates, totalling 2660 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('featureEngineering',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('standardScale',
                                                                         StandardScaler(),
                                                                         ['size',
                                                                          'vac',
                                                                          'ci',
                                                                          'ce',
                                                                          'refgq',
                                                                          'altgq']),
                                                                        ('oneHot',
                     

In [33]:
grid.best_params_

{'classifier__selected_model': ('gradientBoosting',
  {'learning_rate': 0.1,
   'loss': 'deviance',
   'max_depth': 5,
   'max_features': None,
   'n_estimators': 125,
   'subsample': 0.5})}

In [36]:
pd.DataFrame(grid.cv_results_).sort_values("rank_test_score")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__selected_model,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
131,0.330136,0.006551,0.009375,0.001492,"(gradientBoosting, {'learning_rate': 0.1, 'los...",{'classifier__selected_model': ('gradientBoost...,0.991597,0.985994,0.988827,0.985994,0.980282,0.986539,0.003757,1
140,0.325928,0.004566,0.009375,0.000798,"(gradientBoosting, {'learning_rate': 0.1, 'los...",{'classifier__selected_model': ('gradientBoost...,0.991597,0.985994,0.988827,0.985994,0.980282,0.986539,0.003757,1
50,0.145708,0.008526,0.016163,0.002309,"(randomForest, {'class_weight': 'balanced', 'c...",{'classifier__selected_model': ('randomForest'...,0.988827,0.988764,0.980609,0.988827,0.983333,0.986072,0.003457,3
67,0.250324,0.010273,0.018551,0.002581,"(randomForest, {'class_weight': 'balanced', 'c...",{'classifier__selected_model': ('randomForest'...,0.988827,0.988827,0.980609,0.991597,0.980392,0.986050,0.004643,4
66,0.198672,0.007811,0.017358,0.002057,"(randomForest, {'class_weight': 'balanced', 'c...",{'classifier__selected_model': ('randomForest'...,0.988827,0.994382,0.980609,0.988827,0.977528,0.986035,0.006117,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,0.025468,0.003972,0.010395,0.003112,"(svm, {'C': 0.5, 'decision_function_shape': 'o...","{'classifier__selected_model': ('svm', {'C': 0...",0.944444,0.931034,0.939828,0.946779,0.912281,0.934873,0.012514,527
418,0.021742,0.000978,0.009175,0.001597,"(svm, {'C': 0.5, 'decision_function_shape': 'o...","{'classifier__selected_model': ('svm', {'C': 0...",0.969014,0.893983,0.951289,0.946479,0.905028,0.933159,0.028698,529
424,0.020478,0.002538,0.009676,0.000901,"(svm, {'C': 0.5, 'decision_function_shape': 'o...","{'classifier__selected_model': ('svm', {'C': 0...",0.969014,0.893983,0.951289,0.946479,0.905028,0.933159,0.028698,529
448,0.018940,0.002367,0.009375,0.001016,"(svm, {'C': 2, 'decision_function_shape': 'ovr...","{'classifier__selected_model': ('svm', {'C': 2...",0.944751,0.903409,0.948864,0.949438,0.916427,0.932578,0.019023,531


In [45]:
trainPredictions = grid.predict(trainData)
np.average(trainPredictions == trainTarget)

1.0

In [46]:
testPredictions = grid.predict(testData)
np.average(testPredictions == testTarget)

0.9694444444444444

In [51]:
import seaborn as sn
from sklearn.metrics import confusion_matrix

confusionMatrixTest = confusion_matrix(testTarget, testPredictions)
fig, ax = plt.subplots()
sn.heatmap(confusionMatrixTest, annot=True, cmap = "RdYlGn")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

While the accuracy of 100% in the train class indicates overfitting (which is rare for decision trees and forests), the accuracy of 96.9% in test cases is satisfactory for me. We can also see that our model favors false negatives a bit more than false positives. Seeing that the features are not very numerous, it makes sense to me that decision trees, or more precisely gradient boosting forests, led to the best seen results.

### Saving the model for later
In case we do not want to retrain the model later, the following cell saves it to disk:

In [58]:
import lzma
import pickle

model = grid.best_estimator_
with lzma.open("./model.pickle", "wb") as model_file:
    pickle.dump(model, model_file)

To load the model, we can use the following cell:

In [59]:
import lzma
import pickle
with lzma.open("./model.pickle", "rb") as model_file:
    model = pickle.load(model_file)

## Step 4 - executing the pipeline on the unknown dataset
Finally we have our trained pipeline, which can be used to classify the unknown dataset. In this step we will do just that and save the results into a file.

In [62]:
unknownPredictions = model.predict(unknownDataset)

In [75]:
namedPredictions = pd.DataFrame({
    "id": unknownIds,
    "prediction": unknownPredictions
})
namedPredictions

,id,prediction
0,DEL00000012,0.0
1,DEL00000018,0.0
2,DEL00000020,1.0
3,DEL00000080,1.0
4,DEL00000114,1.0
...,...,...
27100,DEL00202501,0.0
27101,DEL00202508,1.0
27102,DEL00202560,1.0
27103,DEL00202579,1.0


In [76]:
namedPredictions.to_csv('./results.tsv', sep='\t')